In [29]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from houseprices',con=engine)

engine.dispose()

In [2]:
numeric_columns = df.select_dtypes(['int64', 'float64']).columns
np.abs(df[numeric_columns].iloc[:,1:].corr().loc[:,"saleprice"]).sort_values(ascending=False)


saleprice        1.000000
overallqual      0.790982
grlivarea        0.708624
garagecars       0.640409
garagearea       0.623431
totalbsmtsf      0.613581
firstflrsf       0.605852
fullbath         0.560664
totrmsabvgrd     0.533723
yearbuilt        0.522897
yearremodadd     0.507101
garageyrblt      0.486362
masvnrarea       0.477493
fireplaces       0.466929
bsmtfinsf1       0.386420
lotfrontage      0.351799
wooddecksf       0.324413
secondflrsf      0.319334
openporchsf      0.315856
halfbath         0.284108
lotarea          0.263843
bsmtfullbath     0.227122
bsmtunfsf        0.214479
bedroomabvgr     0.168213
kitchenabvgr     0.135907
enclosedporch    0.128578
screenporch      0.111447
poolarea         0.092404
mssubclass       0.084284
overallcond      0.077856
mosold           0.046432
threessnporch    0.044584
yrsold           0.028923
lowqualfinsf     0.025606
miscval          0.021190
bsmthalfbath     0.016844
bsmtfinsf2       0.011378
Name: saleprice, dtype: float64

In [3]:
df.housestyle = df.housestyle.apply(str.strip)

for i in range (0, len(df.housestyle)):
        if df.housestyle[i] == '1.5Fin':
            df.housestyle[i] = 0
        elif df.housestyle[i] == '1.5Unf':
            df.housestyle[i] = 1
        elif df.housestyle[i] == '1Story':
            df.housestyle[i] = 2
        elif df.housestyle[i] == '2.5Fin':
            df.housestyle[i] = 3
        elif df.housestyle[i] == '2.5Unf':
            df.housestyle[i] = 4
        elif df.housestyle[i] == '2Story':
            df.housestyle[i] = 5
        elif df.housestyle[i] == 'SFoyer':
            df.housestyle[i] = 6
        elif df.housestyle[i] == 'SLv1':
            df.housestyle[i] = 7
        else:
            df.housestyle[i] = 8

In [4]:
df["is_street_paved"] = pd.get_dummies(df.street, drop_first=True)

df.extercond = df.extercond.apply(str.strip)
np.unique(df.extercond)

for i in range (0, len(df.extercond)):
    if df.extercond[i] == 'Ex':
        df.extercond[i] = 0
    elif df.extercond[i] == 'Fa':
        df.extercond[i] = 1
    elif df.extercond[i] == 'Gd':
        df.extercond[i] = 2
    elif df.extercond[i] == 'Po':
        df.extercond[i] = 3
    else:
        df.extercond[i] = 4


In [5]:
df.foundation = df.foundation.apply(str.strip)
np.unique(df.foundation)

for i in range (0, len(df.foundation)):
    if df.foundation[i] == 'BrkTil':
        df.foundation[i] = 0
    elif df.foundation[i] == 'CBlock':
        df.foundation[i] = 1
    elif df.foundation[i] == 'PConc':
        df.foundation[i] = 2
    elif df.foundation[i] == 'Slab':
        df.foundation[i] = 3
    elif df.foundation[i] == 'Stone':
        df.foundation[i] = 4
    else:
        df.foundation[i] = 5

In [6]:
df.functional = df.functional.apply(str.strip)
np.unique(df.functional)

for i in range (0, len(df.functional)):
    if df.functional[i] == 'Typ':
        df.functional[i] = 0
    elif df.functional[i] == 'Min1':
        df.functional[i] = 1
    elif df.functional[i] == 'Min2':
        df.functional[i] = 2
    elif df.functional[i] == 'Mod':
        df.functional[i] = 3
    elif df.functional[i] == 'Maj1':
        df.functional[i] = 4
    elif df.functional[i] == 'Maj2':
        df.functional[i] = 5
    elif df.functional[i] == 'Sev':
        df.functional[i] = 6
    else:
        df.functional[i] = 7
        

In [8]:
X = df[['housestyle', 'is_street_paved', 'overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf']]
df.housestyle = df['housestyle'].astype(str).astype(int)
df.extercond = df['extercond'].astype(str).astype(int)
df.foundation = df['foundation'].astype(str).astype(int)
df.functional = df['functional'].astype(str).astype(int)

In [9]:
Y = df.saleprice

In [10]:
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.761
Model:                            OLS   Adj. R-squared:                  0.760
Method:                 Least Squares   F-statistic:                     661.1
Date:                Mon, 09 Mar 2020   Prob (F-statistic):               0.00
Time:                        18:59:19   Log-Likelihood:                -17499.
No. Observations:                1460   AIC:                         3.501e+04
Df Residuals:                    1452   BIC:                         3.506e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const           -1.086e+05   1.63e+04     -6.647      0.000   -1.41e+05   -7.66e+04
housestyle        147.7373    562.471      0.263      0.793    -955.605    1251.079
is_street_paved  9618.8357    1.6e+04      0.600      0.549   -2.18e+04    4.11e+04
overallqual      2.354e+04   1088.980     21.621      0.000    2.14e+04    2.57e+04
grlivarea          45.1535      2.550     17.706      0.000      40.151      50.156
garagecars       1.447e+04   3028.634      4.778      0.000    8529.314    2.04e+04
garagearea         17.5354     10.503      1.670      0.095      -3.067      38.137
totalbsmtsf        31.8095      3.111     10.225      0.000      25.707      37.912
==============================================================================
Omnibus:                      440.091   Durbin-Watson:                   1.972
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            41191.406
Skew:                          -0.345   Prob(JB):                         0.00
Kurtosis:                      29.012   Cond. No.                     4.45e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.45e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [26]:
alphas = [np.power(10.0,p) for p in np.arange(-10,40,1)]
X_train,X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [33]:
# ridge regression
ridge_cv = RidgeCV(alphas)
ridge_cv.fit(X_train, y_train)
y_preds_train = ridge_cv.predict(X_train)
y_preds_test = ridge_cv.predict(X_test)

print("Best alpha value is: {}".format(ridge_cv.alpha_))
print("R-squared of the model in training set is: {}".format(ridge_cv.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(ridge_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

Best alpha value is: 10.0
R-squared of the model in training set is: 0.7525084701311258
-----Test set statistics-----
R-squared of the model in test set is: 0.796058957443962
Mean absolute error of the prediction is: 23496.409442561446
Mean squared error of the prediction is: 1225365770.2194555
Root mean squared error of the prediction is: 35005.22489885554
Mean absolute percentage error of the prediction is: 15.231976675222144


In [37]:
# lasso regression
lasso_cv = LassoCV(alphas = alphas, cv=5)
lasso_cv.fit(X_train, y_train)
y_preds_train = lasso_cv.predict(X_train)
y_preds_test = lasso_cv.predict(X_test)

print("Best alpha value is: {}".format(lasso_cv.alpha_))
print("R-squared of the model in training set is: {}".format(lasso_cv.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lasso_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

Best alpha value is: 100.0
R-squared of the model in training set is: 0.7524861465154333
-----Test set statistics-----
R-squared of the model in test set is: 0.7958597833952105
Mean absolute error of the prediction is: 23536.567962318015
Mean squared error of the prediction is: 1226562493.8342667
Root mean squared error of the prediction is: 35022.31422727896
Mean absolute percentage error of the prediction is: 15.271121021202239


In [38]:
# elastic net regression
elasticnet_cv = ElasticNetCV(alphas=alphas, cv=5)

elasticnet_cv.fit(X_train, y_train)

y_preds_train = elasticnet_cv.predict(X_train)
y_preds_test = elasticnet_cv.predict(X_test)

print("Best alpha value is: {}".format(elasticnet_cv.alpha_))
print("R-squared of the model in training set is: {}".format(elasticnet_cv.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(elasticnet_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

Best alpha value is: 0.1
R-squared of the model in training set is: 0.7520101259529267
-----Test set statistics-----
R-squared of the model in test set is: 0.7970846754665581
Mean absolute error of the prediction is: 23267.262375233837
Mean squared error of the prediction is: 1219202813.8128695
Root mean squared error of the prediction is: 34917.084841276046
Mean absolute percentage error of the prediction is: 15.00410507942702


In [40]:
#ols linear regression

lrm=LinearRegression()
lrm.fit(X_train, y_train)

y_preds_train = lrm.predict(X_train)
y_preds_test = lrm.predict(X_test)

print("R-squared of the model in training set is: {}".format(lrm.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lrm.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in training set is: 0.7525526517388614
-----Test set statistics-----
R-squared of the model in test set is: 0.7957916179036378
Mean absolute error of the prediction is: 23536.93696415532
Mean squared error of the prediction is: 1226972061.51636
Root mean squared error of the prediction is: 35028.160978223794
Mean absolute percentage error of the prediction is: 15.270756637564787


All of the models have very close scores to eachother. The model with the highest R^2 score is elastic net regression model. It also has the lowest test statistics making it the most accurate.